In [ ]:
pip install fastapi
pip install opencv-python
pip install Pillow
pip install timm
pip install python-multipart
pip install uvicorn
pip install nest-asyncio
pip install pyngrok

SyntaxError: invalid syntax (1695741726.py, line 1)

In [1]:
from typing import List

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse, StreamingResponse

import pickle
import cv2
import io
import numpy as np

import os

import torch
import cv2
from PIL import Image

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

In [2]:
user_dir = os.getcwd()
user_dir
file = 'model.pkl'
model_path = os.path.join(user_dir, file)


In [7]:
from torch import nn
class MRIClassifer(nn.Module):
    def __init__(self, num_classes=53):
        super(MRIClassifer, self).__init__()
        # Where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 1280
        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

def preprocess(image):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.PILToTensor(),
    ])
    return transform(image)


model_pkl = open(model_path, 'rb')
model = pickle.load(model_pkl)
model_pkl.close()

In [13]:
device = torch.device('cpu')

In [ ]:
app = FastAPI()

@app.post("/uploadfiles/")
async def create_upload_files(files: List[UploadFile] = File(...)):
    # Return preprocessed input batch and loaded image
        original_image, image_tensor = pre_process_image(image, transform)

        # Run the model and postpocess the output
        prediction = predict(model, image_tensor, device)

        # # Post process and stitch together the two images to return them
        
            
        return prediction

In [ ]:
auth_token = "" #@param {type:"string"}
# This tutorial uses ngrok to deploy the website, we probably wont do that, but here is the code in case we do.

# Authenticate ngrok
# https://dashboard.ngrok.com/signup
# Then go to the "Your Authtoken" tab in the sidebar and copy the API key
import os
os.system(f"ngrok authtoken {auth_token}")

In [ ]:
from pyngrok import ngrok

# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)

In [15]:
import nest_asyncio

# Allow for asyncio to work within the Jupyter notebook cell
nest_asyncio.apply()

import uvicorn

# Run the FastAPI app using uvicorn
uvicorn.run(app)

INFO:     Started server process [74356]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:61321 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:61321 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [74356]
